## Load functionality

In [126]:
from transformers import AutoTokenizer, AutoModel
from pathlib import Path
import tensorflow as tf
import polars as pl
import datetime

from ebrec.utils._constants import *

from ebrec.utils._behaviors import (
    create_binary_labels_column,
    sampling_strategy_wu2019,
    add_prediction_scores,
    truncate_history,
    ebnerd_from_path,
)
from ebrec.evaluation import MetricEvaluator, AucScore, NdcgScore, MrrScore
from ebrec.utils._articles import convert_text2encoding_with_transformers
from ebrec.utils._polars import concat_str_columns, slice_join_dataframes
from ebrec.utils._articles import create_article_id_to_value_mapping
from ebrec.utils._nlp import get_transformers_word_embeddings
from ebrec.utils._python import write_submission_file, rank_predictions_by_score

from ebrec.models.newsrec.dataloader import NRMSDataLoader
from ebrec.models.newsrec.model_config import hparams_nrms
from ebrec.models.newsrec import NRMSModel

In [127]:
# List all physical devices
gpus = tf.config.experimental.list_physical_devices("GPU")
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

physical_devices = tf.config.list_physical_devices()
print("Available devices:", physical_devices)

Available devices: [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]


## Load dataset

### Generate labels
We sample a few just to get started. For testset we just make up a dummy column with 0 and 1 - this is not the true labels.

In [128]:
# PATH = Path("~/ebnerd_data").expanduser()
# #
# DATASPLIT = "ebnerd_small"
# DUMP_DIR = Path("ebnerd_predictions")
# DUMP_DIR.mkdir(exist_ok=True, parents=True)

In [129]:
from pathlib import Path

# Use raw string to avoid issues with backslashes
PATH = Path(r"C:\Users\antot\Downloads\ebnerd-benchmark\examples\ebnerd_data").expanduser()
TRAIN = f"ebnerd_small"  # [ebnerd_demo, ebnerd_small, ebnerd_large]
VAL = f"ebnerd_small"
TEST = f"ebnerd_testset"#, "ebnerd_testset_gt"


# Create a directory for dumping predictions
#DUMP_DIR = Path("ebnerd_predictions")
#DUMP_DIR.mkdir(exist_ok=True, parents=True)

In [130]:
DUMP_DIR = Path(r"C:\Users\antot\Downloads\ebnerd-benchmark\examples").expanduser()
DUMP_DIR.mkdir(exist_ok=True, parents=True)

History size can often be a memory bottleneck; if adjusted, the NRMS hyperparameter ```history_size``` must be updated to ensure compatibility and efficient memory usage

In [131]:
HISTORY_SIZE = 20
hparams_nrms.history_size = HISTORY_SIZE

In [132]:
# We just want to load the necessary columns
COLUMNS = [
    DEFAULT_USER_COL,
    DEFAULT_IMPRESSION_ID_COL,
    DEFAULT_IMPRESSION_TIMESTAMP_COL,
    DEFAULT_HISTORY_ARTICLE_ID_COL,
    DEFAULT_CLICKED_ARTICLES_COL,
    DEFAULT_INVIEW_ARTICLES_COL,
]
# This notebook is just a simple 'get-started'; we down sample the number of samples to just run quickly through it.
FRACTION = 0.01

In this example we sample the dataset, just to keep it smaller. We'll split the training data into training and validation 

In [133]:
# Load your train and validation datasets directly
df_train = ebnerd_from_path(
    PATH.joinpath("ebnerd_small/train"),
    history_size=HISTORY_SIZE,
    
).select(COLUMNS).pipe(
    sampling_strategy_wu2019,
    npratio=4,
    shuffle=True,
    with_replacement=True,
    seed=123,
).pipe(create_binary_labels_column)

df_validation = ebnerd_from_path(
    PATH.joinpath("ebnerd_small/validation"),
    history_size=HISTORY_SIZE
    
).select(COLUMNS).pipe(
    sampling_strategy_wu2019,
    npratio=4,
    shuffle=True,
    with_replacement=True,
    seed=123,
).pipe(create_binary_labels_column)

print(f"Train samples: {df_train.height}\nValidation samples: {df_validation.height}")

# Preview the datasets
print("Train Data Sample:")
print(df_train.head(2))

print("Validation Data Sample:")
print(df_validation.head(2))


Train samples: 234277
Validation samples: 246289
Train Data Sample:
shape: (2, 7)
┌─────────┬──────────────┬──────────────┬──────────────┬──────────────┬──────────────┬─────────────┐
│ user_id ┆ impression_i ┆ impression_t ┆ article_id_f ┆ article_ids_ ┆ article_ids_ ┆ labels      │
│ ---     ┆ d            ┆ ime          ┆ ixed         ┆ clicked      ┆ inview       ┆ ---         │
│ u32     ┆ ---          ┆ ---          ┆ ---          ┆ ---          ┆ ---          ┆ list[i8]    │
│         ┆ u32          ┆ datetime[μs] ┆ list[i32]    ┆ list[i64]    ┆ list[i64]    ┆             │
╞═════════╪══════════════╪══════════════╪══════════════╪══════════════╪══════════════╪═════════════╡
│ 139836  ┆ 149474       ┆ 2023-05-24   ┆ [0, 9745590, ┆ [9778657]    ┆ [9778728,    ┆ [0, 0, … 1] │
│         ┆              ┆ 07:47:53     ┆ … 9765156]   ┆              ┆ 9778669, …   ┆             │
│         ┆              ┆              ┆              ┆              ┆ 9778657]     ┆             │
│ 143471 

In [134]:

#print(f"Model Directory: {MODEL_NAME}")

# Data preprocessing parameters
MAX_TITLE_LENGTH = 30
HISTORY_SIZE = 20
FRACTION = 1.0
EPOCHS = 5
FRACTION_TEST = 1.0
hparams_nrms.history_size = HISTORY_SIZE

# Batch sizes
BATCH_SIZE_TRAIN = 64
BATCH_SIZE_VAL = 64
BATCH_SIZE_TEST_WO_B = 64
BATCH_SIZE_TEST_W_B = 64
N_CHUNKS_TEST = 10
CHUNKS_DONE = 0

# We just want to load the necessary columns
COLUMNS = [
    DEFAULT_USER_COL,
    DEFAULT_IMPRESSION_ID_COL,
    DEFAULT_IMPRESSION_TIMESTAMP_COL,
    DEFAULT_HISTORY_ARTICLE_ID_COL,
    DEFAULT_CLICKED_ARTICLES_COL,
    DEFAULT_INVIEW_ARTICLES_COL,
]
# This notebook is just a simple 'get-started'; we down sample the number of samples to just run quickly through it.
FRACTION = 0.01


### Test set
We'll use the validation set, as the test set.

In [135]:
df_test = (
    ebnerd_from_path(
        PATH.joinpath(PATH, "ebnerd_testset/test")
    )
    .sample(fraction=FRACTION)
)

print(f"Test samples: {df_test.height}")
print("Test Data Sample:")
print(df_test.head(2))


Test samples: 135367
Test Data Sample:
shape: (2, 15)
┌───────────┬───────────┬───────────┬───────────┬───┬───────────┬───────────┬───────────┬──────────┐
│ impressio ┆ impressio ┆ read_time ┆ scroll_pe ┆ … ┆ is_subscr ┆ session_i ┆ is_beyond ┆ article_ │
│ n_id      ┆ n_time    ┆ ---       ┆ rcentage  ┆   ┆ iber      ┆ d         ┆ _accuracy ┆ id_fixed │
│ ---       ┆ ---       ┆ f32       ┆ ---       ┆   ┆ ---       ┆ ---       ┆ ---       ┆ ---      │
│ u32       ┆ datetime[ ┆           ┆ f32       ┆   ┆ bool      ┆ u32       ┆ bool      ┆ list[i32 │
│           ┆ μs]       ┆           ┆           ┆   ┆           ┆           ┆           ┆ ]        │
╞═══════════╪═══════════╪═══════════╪═══════════╪═══╪═══════════╪═══════════╪═══════════╪══════════╡
│ 49308622  ┆ 2023-06-0 ┆ 207.0     ┆ null      ┆ … ┆ true      ┆ 6165783   ┆ false     ┆ [9788125 │
│           ┆ 7         ┆           ┆           ┆   ┆           ┆           ┆           ┆ ,        │
│           ┆ 18:45:35  ┆           ┆

In [136]:
COLUMNSTEST = [
    DEFAULT_USER_COL,
    DEFAULT_IMPRESSION_ID_COL,
    DEFAULT_IMPRESSION_TIMESTAMP_COL,
    DEFAULT_INVIEW_ARTICLES_COL,
    DEFAULT_LABELS_COL

]

## Load articles

In [137]:

df_articles_train = pl.read_parquet(PATH.joinpath("ebnerd_small/articles.parquet"))
df_articles_train.head()
#df_articles_test = pl.read_parquet(TEST_MAIN_PATH.joinpath("articles.parquet"))

article_id,title,subtitle,last_modified_time,premium,body,published_time,image_ids,article_type,url,ner_clusters,entity_groups,topics,category,subcategory,category_str,total_inviews,total_pageviews,total_read_time,sentiment_score,sentiment_label
i32,str,str,datetime[μs],bool,str,datetime[μs],list[i64],str,str,list[str],list[str],list[str],i16,list[i16],str,i32,i32,f32,f32,str
3001353,"""Natascha var i…","""Politiet frygt…",2023-06-29 06:20:33,false,"""Sagen om den ø…",2006-08-31 08:06:45,[3150850],"""article_defaul…","""https://ekstra…",[],[],"[""Kriminalitet"", ""Personfarlig kriminalitet""]",140,[],"""krimi""",null,null,null,0.9955,"""Negative"""
3003065,"""Kun Star Wars …","""Biografgængern…",2023-06-29 06:20:35,false,"""Vatikanet har …",2006-05-21 16:57:00,[3006712],"""article_defaul…","""https://ekstra…",[],[],"[""Underholdning"", ""Film og tv"", ""Økonomi""]",414,"[433, 434]","""underholdning""",null,null,null,0.846,"""Positive"""
3012771,"""Morten Bruun f…","""FODBOLD: Morte…",2023-06-29 06:20:39,false,"""Kemien mellem …",2006-05-01 14:28:40,[3177953],"""article_defaul…","""https://ekstra…",[],[],"[""Erhverv"", ""Kendt"", … ""Ansættelsesforhold""]",142,"[196, 199]","""sport""",null,null,null,0.8241,"""Negative"""
3023463,"""Luderne flytte…","""I landets tynd…",2023-06-29 06:20:43,false,"""Det frække erh…",2007-03-24 08:27:59,[3184029],"""article_defaul…","""https://ekstra…",[],[],"[""Livsstil"", ""Erotik""]",118,[133],"""nyheder""",null,null,null,0.7053,"""Neutral"""
3032577,"""Cybersex: Hvor…","""En flirtende s…",2023-06-29 06:20:46,false,"""De fleste af o…",2007-01-18 10:30:37,[3030463],"""article_defaul…","""https://ekstra…",[],[],"[""Livsstil"", ""Partnerskab""]",565,[],"""sex_og_samliv""",null,null,null,0.9307,"""Neutral"""


In [138]:
df_articles_test = pl.read_parquet(PATH.joinpath(PATH, "ebnerd_testset/articles.parquet"))
df_articles_test.head()


article_id,title,subtitle,last_modified_time,premium,body,published_time,image_ids,article_type,url,ner_clusters,entity_groups,topics,category,subcategory,category_str,total_inviews,total_pageviews,total_read_time,sentiment_score,sentiment_label
i32,str,str,datetime[μs],bool,str,datetime[μs],list[i64],str,str,list[str],list[str],list[str],i16,list[i16],str,i32,i32,f32,f32,str
3000022,"""Hanks beskyldt…","""Tom Hanks har …",2023-06-29 06:20:32,false,"""Tom Hanks skul…",2006-09-20 09:24:18,[3518381],"""article_defaul…","""https://ekstra…","[""David Gardner""]","[""PER""]","[""Kriminalitet"", ""Kendt"", … ""Litteratur""]",414,[432],"""underholdning""",null,null,null,0.9911,"""Negative"""
3000063,"""Bostrups aske …","""Studieværten b…",2023-06-29 06:20:32,false,"""Strålende sens…",2006-09-24 07:45:30,"[3170935, 3170939]","""article_defaul…","""https://ekstra…",[],[],"[""Kendt"", ""Underholdning"", … ""Personlig begivenhed""]",118,[133],"""nyheder""",null,null,null,0.5155,"""Neutral"""
3000613,"""Jesper Olsen r…","""Den tidligere …",2023-06-29 06:20:33,false,"""Jesper Olsen, …",2006-05-09 11:29:00,[3164998],"""article_defaul…","""https://ekstra…","[""Frankrig"", ""Jesper Olsen"", … ""Jesper Olsen""]","[""LOC"", ""PER"", … ""PER""]","[""Kendt"", ""Sport"", … ""Sygdom og behandling""]",142,"[196, 271]","""sport""",null,null,null,0.9876,"""Negative"""
3000700,"""Madonna topløs…","""47-årige Madon…",2023-06-29 06:20:33,false,"""Skal du have s…",2006-05-04 11:03:12,[3172046],"""article_defaul…","""https://ekstra…",[],[],"[""Kendt"", ""Livsstil"", ""Underholdning""]",414,[432],"""underholdning""",null,null,null,0.8786,"""Neutral"""
3000840,"""Otto Brandenbu…","""Sangeren og sk…",2023-06-29 06:20:33,false,"""'Og lidt for S…",2007-03-01 18:34:00,[3914446],"""article_defaul…","""https://ekstra…",[],[],"[""Kendt"", ""Underholdning"", … ""Musik og lyd""]",118,[133],"""nyheder""",null,null,null,0.9468,"""Negative"""


In [139]:
BATCH_SIZE = 8 # try with 64
df_train_subset = df_train[:1000] 
df_val_subset = df_validation[:1000]  

In [140]:
from transformers import AutoModel, AutoTokenizer
TEXT_COLUMNS_TO_USE = [DEFAULT_SUBTITLE_COL, DEFAULT_TITLE_COL]
MAX_TITLE_LENGTH = 30

from transformers import AutoModel, AutoTokenizer
import pandas as pd

# Define Available Models
MODEL_CLASSES = {
    "xlm-roberta": "FacebookAI/xlm-roberta-base",
    "bert": "bert-base-uncased",
    "roberta": "roberta-base",
}

# Select Model
SELECTED_MODEL = "bert"  # Change this to "bert" or "roberta" as needed
TRANSFORMER_MODEL_NAME = MODEL_CLASSES[SELECTED_MODEL]

TEXT_COLUMNS_TO_USE = [DEFAULT_SUBTITLE_COL, DEFAULT_TITLE_COL]
MAX_TITLE_LENGTH = 30

# Load Transformer Model and Tokenizer
transformer_model = AutoModel.from_pretrained(TRANSFORMER_MODEL_NAME)
transformer_tokenizer = AutoTokenizer.from_pretrained(TRANSFORMER_MODEL_NAME)

# Initialize Word Embeddings
word2vec_embedding = get_transformers_word_embeddings(transformer_model)

# Process Training Data
df_articles_train, cat_cal = concat_str_columns(df_articles_train, columns=TEXT_COLUMNS_TO_USE)
df_articles_train, token_col_title = convert_text2encoding_with_transformers(
    df_articles_train, transformer_tokenizer, cat_cal, max_length=MAX_TITLE_LENGTH
)
article_mapping_train = create_article_id_to_value_mapping(
    df=df_articles_train, value_col=token_col_title
)

# Process Test Data
df_articles_test, cat_cal = concat_str_columns(df_articles_test, columns=TEXT_COLUMNS_TO_USE)
df_articles_test, token_col_title = convert_text2encoding_with_transformers(
    df_articles_test, transformer_tokenizer, cat_cal, max_length=MAX_TITLE_LENGTH
)
article_mapping_test = create_article_id_to_value_mapping(
    df=df_articles_test, value_col=token_col_title
)

In [141]:
df_val_subset

user_id,impression_id,impression_time,article_id_fixed,article_ids_clicked,article_ids_inview,labels
u32,u32,datetime[μs],list[i32],list[i64],list[i64],list[i8]
22548,96791,2023-05-28 04:21:24,"[9773295, 9769504, … 9776929]",[9784696],"[9784710, 9784591, … 9784696]","[0, 0, … 1]"
22548,96798,2023-05-28 04:31:48,"[9773295, 9769504, … 9776929]",[9784281],"[9782656, 9783405, … 9784281]","[0, 0, … 1]"
22548,96801,2023-05-28 04:30:17,"[9773295, 9769504, … 9776929]",[9784444],"[9784444, 9783405, … 9782108]","[1, 0, … 0]"
22548,96808,2023-05-28 04:27:19,"[9773295, 9769504, … 9776929]",[9781983],"[9784406, 9781983, … 9695098]","[0, 1, … 0]"
22548,96810,2023-05-28 04:29:47,"[9773295, 9769504, … 9776929]",[9784642],"[9782108, 9784642, … 9782108]","[0, 1, … 0]"
22548,96818,2023-05-28 04:18:42,"[9773295, 9769504, … 9776929]",[9782884],"[9784710, 9784710, … 9784710]","[0, 0, … 0]"
22548,96821,2023-05-28 04:30:58,"[9773295, 9769504, … 9776929]",[9784160],"[9784642, 9784662, … 9784702]","[0, 0, … 0]"
22548,96824,2023-05-28 04:20:28,"[9773295, 9769504, … 9776929]",[9784679],"[9784591, 9784696, … 9784679]","[0, 0, … 1]"
25554,97411,2023-05-28 04:10:48,"[9779489, 9779541, … 9780019]",[9784591],"[9784679, 9784710, … 9784679]","[0, 0, … 0]"


In [142]:
from transformers import AutoModel, AutoTokenizer
TEXT_COLUMNS_TO_USE = [DEFAULT_SUBTITLE_COL, DEFAULT_TITLE_COL]
MAX_TITLE_LENGTH = 30

from transformers import AutoModel, AutoTokenizer
import pandas as pd

# Define Available Models
MODEL_CLASSES = {
    "xlm-roberta": "FacebookAI/xlm-roberta-base",
    "bert": "bert-base-uncased",
    "roberta": "roberta-base",
}

# Select Model
SELECTED_MODEL = "bert"  # Change this to "bert" or "roberta" as needed
TRANSFORMER_MODEL_NAME = MODEL_CLASSES[SELECTED_MODEL]

TEXT_COLUMNS_TO_USE = [DEFAULT_SUBTITLE_COL, DEFAULT_TITLE_COL]
MAX_TITLE_LENGTH = 30

# Load Transformer Model and Tokenizer
transformer_model = AutoModel.from_pretrained(TRANSFORMER_MODEL_NAME)
transformer_tokenizer = AutoTokenizer.from_pretrained(TRANSFORMER_MODEL_NAME)

# Initialize Word Embeddings
word2vec_embedding = get_transformers_word_embeddings(transformer_model)

# Process Training Data
df_articles_train, cat_cal = concat_str_columns(df_articles_train, columns=TEXT_COLUMNS_TO_USE)
df_articles_train, token_col_title = convert_text2encoding_with_transformers(
    df_articles_train, transformer_tokenizer, cat_cal, max_length=MAX_TITLE_LENGTH
)
article_mapping_train = create_article_id_to_value_mapping(
    df=df_articles_train, value_col=token_col_title
)

# Process Test Data
df_articles_test, cat_cal = concat_str_columns(df_articles_test, columns=TEXT_COLUMNS_TO_USE)
df_articles_test, token_col_title = convert_text2encoding_with_transformers(
    df_articles_test, transformer_tokenizer, cat_cal, max_length=MAX_TITLE_LENGTH
)
article_mapping_test = create_article_id_to_value_mapping(
    df=df_articles_test, value_col=token_col_title
)

Glove intergration

# Initiate the dataloaders
In the implementations we have disconnected the models and data. Hence, you should built a dataloader that fits your needs.

Note, with this ```NRMSDataLoader``` the ```eval_mode=False``` is meant for ```model.model.fit()``` whereas ```eval_mode=True``` is meant for ```model.scorer.predict()```. 

In [143]:
# Initialize DataLoaders for train and validation
print("Initializing train and validation dataloaders...")

Initializing train and validation dataloaders...


In [144]:
BATCH_SIZE = 8 # try with 64
df_train_subset = df_train[:1000] 
df_val_subset = df_validation[:1000]  
train_dataloader = NRMSDataLoader(
    behaviors=df_train_subset,
    article_dict=article_mapping_train,
    unknown_representation="zeros",
    history_column=DEFAULT_HISTORY_ARTICLE_ID_COL,
    eval_mode=False,
    batch_size=BATCH_SIZE,
)
val_dataloader = NRMSDataLoader(
    behaviors=df_val_subset,
    article_dict=article_mapping_train,
    unknown_representation="zeros",
    history_column=DEFAULT_HISTORY_ARTICLE_ID_COL,
    eval_mode=False,
    batch_size=BATCH_SIZE,
)

## Train the model


In [145]:
# List all physical devices
physical_devices = tf.config.list_physical_devices()
print("Available devices:", physical_devices)

Available devices: [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]


Initiate the NRMS-model:

In [146]:
model = NRMSModel(
    hparams=hparams_nrms,
    word2vec_embedding=word2vec_embedding,
    seed=42,
)
model.model.compile(
    optimizer=model.model.optimizer,
    loss=model.model.loss,
    metrics=["AUC"],
)

MODEL_NAME = model.__class__.__name__
MODEL_WEIGHTS = DUMP_DIR.joinpath(f"state_dict/{MODEL_NAME}/weights")
LOG_DIR = DUMP_DIR.joinpath(f"runs/{MODEL_NAME}")

In [147]:
from pathlib import Path
from tensorflow.keras.callbacks import ModelCheckpoint

# Define paths
DUMP_DIR = Path(r"C:\Users\antot\Downloads\ebnerd-benchmark\examples").expanduser()
DUMP_DIR.mkdir(exist_ok=True, parents=True)

MODEL_NAME = model.__class__.__name__
MODEL_WEIGHTS = DUMP_DIR.joinpath(f"state_dict/{MODEL_NAME}/weights")
LOG_DIR = DUMP_DIR.joinpath(f"runs/{MODEL_NAME}")

# Ensure directory for weights exists
MODEL_WEIGHTS.parent.mkdir(parents=True, exist_ok=True)

# Compile the model
# model = NRMSModel(
#     hparams=hparams_nrms,
#     word2vec_embedding=word2vec_embedding,
#     seed=42,
# )
# model.model.compile(
#     optimizer=model.model.optimizer,
#     loss=model.model.loss,
#     metrics=["AUC"],
# )

# Define checkpoint callback
checkpoint_callback = ModelCheckpoint(
    filepath=str(MODEL_WEIGHTS),
    save_weights_only=True,
    monitor='val_auc',
    mode='max',
    save_best_only=True,
    verbose=1
)

In [21]:
# model = NRMSModel
# MODEL_NAME = model.__class__.__name__


## Train and store the weights

In [22]:
import time
import tensorflow as tf
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor="val_auc",
    mode="max",
    patience=3,
    restore_best_weights=True,
)

# Learning rate scheduler
lr_scheduler = tf.keras.callbacks.ReduceLROnPlateau(
    monitor="val_auc",  # Monitor validation AUC
    mode="max",         # Maximize AUC
    factor=0.2,         # Reduce learning rate by 80%
    patience=2,         # Wait for 2 epochs with no improvement
    min_lr=1e-6         # Set a minimum learning rate
)

# Use callbacks if enabled
USE_CALLBACKS = True
callbacks = [lr_scheduler, early_stopping] if USE_CALLBACKS else []

# Training loop
EPOCHS = 5  # Adjust to desired number of epochs
for epoch in range(EPOCHS):
    start_time = time.time()
    print(f"Starting Epoch {epoch + 1}/{EPOCHS}")

    # Train the model for one epoch
    model.model.fit(
        train_dataloader,              # Training data
        validation_data=val_dataloader,  # Validation data
        epochs=1,                       # One epoch at a time
        callbacks=callbacks,            # Use callbacks if enabled
        verbose=1                       # Display progress
    )
    # Measure epoch duration
    epoch_time = time.time() - start_time
    print(f"Epoch {epoch + 1} completed in {epoch_time:.2f} seconds")

# Save weights after training
MODEL_WEIGHTS.parent.mkdir(parents=True, exist_ok=True)  # Ensure directory exists
model.model.save_weights(MODEL_WEIGHTS)
print(f"Model weights saved at: {MODEL_WEIGHTS}")


Starting Epoch 1/5


125/125 [==============================] - 64s 454ms/step - loss: 1.6074 - auc: 0.5484 - val_loss: 1.6088 - val_auc: 0.5039 - lr: 1.0000e-04
Epoch 1 completed in 64.85 seconds
Starting Epoch 2/5
125/125 [==============================] - 60s 480ms/step - loss: 1.5582 - auc: 0.6628 - val_loss: 1.6195 - val_auc: 0.5089 - lr: 1.0000e-04
Epoch 2 completed in 60.54 seconds
Starting Epoch 3/5
125/125 [==============================] - 53s 427ms/step - loss: 1.3822 - auc: 0.7393 - val_loss: 1.6880 - val_auc: 0.5118 - lr: 1.0000e-04
Epoch 3 completed in 53.54 seconds
Starting Epoch 4/5
125/125 [==============================] - 54s 432ms/step - loss: 1.2657 - auc: 0.7879 - val_loss: 1.7387 - val_auc: 0.5122 - lr: 1.0000e-04
Epoch 4 completed in 54.14 seconds
Starting Epoch 5/5
125/125 [==============================] - 53s 425ms/step - loss: 1.1818 - auc: 0.8176 - val_loss: 1.8392 - val_auc: 0.5147 - lr: 1.0000e-04
Epoch 5 completed in 53.32 seconds
Model weights saved at:

In [23]:
if USE_CALLBACKS:
    _ = model.model.load_weights(filepath=MODEL_WEIGHTS)

In [24]:
import gc
from tensorflow.keras.backend import clear_session
from ebrec.utils._polars import split_df_chunks

# Set up validation directories
VAL_DF_DUMP = DUMP_DIR.joinpath("val_predictions", MODEL_NAME)
VAL_DF_DUMP.mkdir(parents=True, exist_ok=True)

# Split validation dataset into manageable chunks
df_val_chunks = split_df_chunks(df_val_subset, n_chunks=32)
df_pred_val = []

# Loop through each chunk
for i, df_val_chunk in enumerate(df_val_chunks):
    print(f"Processing validation chunk: {i + 1}/{len(df_val_chunks)}")

    # Initialize DataLoader for validation set
    val_dataloader = NRMSDataLoader(
        behaviors=df_val_chunk,
        article_dict=article_mapping_train,
        unknown_representation="zeros",
        history_column=DEFAULT_HISTORY_ARTICLE_ID_COL,
        eval_mode=True,
        batch_size=64,  # Batch size for validation
    )

    # Predict scores for validation chunk
    scores = model.scorer.predict(val_dataloader)
    clear_session()

    # Add predictions to the DataFrame
    df_val_chunk = add_prediction_scores(df_val_chunk, scores.tolist()).with_columns(
        pl.col("scores")
        .map_elements(lambda x: list(rank_predictions_by_score(x)))  # Rank predictions
        .alias("ranked_scores")
    )

    # Save predictions for this chunk
    df_val_chunk.select(DEFAULT_IMPRESSION_ID_COL, "ranked_scores").write_parquet(
        VAL_DF_DUMP.joinpath(f"val_pred_chunk_{i + 1}.parquet")
    )

    # Append processed chunk
    df_pred_val.append(df_val_chunk)

    # Cleanup to release memory
    del df_val_chunk, val_dataloader, scores
    gc.collect()

# Combine all validation chunks into a single DataFrame
df_pred_val_combined = pl.concat(df_pred_val)

# Save the full validation predictions
df_pred_val_combined.select(DEFAULT_IMPRESSION_ID_COL, "ranked_scores").write_parquet(
    VAL_DF_DUMP.joinpath("val_predictions_combined.parquet")
)

# Print the first few rows for inspection
print("Validation Predictions:")
print(df_pred_val_combined.head())


Processing validation chunk: 1/32
1/1 [==============================] - 2s 2s/step
Processing validation chunk: 2/32
1/1 [==============================] - 1s 1s/step
Processing validation chunk: 3/32
1/1 [==============================] - 1s 1s/step
Processing validation chunk: 4/32
1/1 [==============================] - 2s 2s/step
Processing validation chunk: 5/32
1/1 [==============================] - 1s 1s/step
Processing validation chunk: 6/32
1/1 [==============================] - 1s 1s/step
Processing validation chunk: 7/32
1/1 [==============================] - 1s 1s/step
Processing validation chunk: 8/32
1/1 [==============================] - 1s 1s/step
Processing validation chunk: 9/32
1/1 [==============================] - 1s 1s/step
Processing validation chunk: 10/32
1/1 [==============================] - 1s 1s/step
Processing validation chunk: 11/32
1/1 [==============================] - 1s 1s/step
Processing validation chunk: 12/32
1/1 [==============================] - 

In [25]:
from ebrec.evaluation.metrics import (
    mean_squared_error,
    accuracy_score,
    roc_auc_score,
    ndcg_score,
    mrr_score,
    log_loss,
    f1_score,
)

evaluator = MetricEvaluator(
    labels=df_pred_val_combined["labels"].to_list(),
    predictions=df_pred_val_combined["scores"].to_list(),
    metric_functions=[
        AucScore(),
        MrrScore(),
        NdcgScore(k=5),
        NdcgScore(k=10),
    ],
)
results = evaluator.evaluate()
print(results)


AUC: 100%|████████████████████████████████| 1000/1000 [00:00<00:00, 7415.88it/s]

<MetricEvaluator class>: 
 {
    "auc": 0.5105,
    "mrr": 0.4893333333333333,
    "ndcg@5": 0.6136776776746045,
    "ndcg@10": 0.6136776776746045
}


Implimentation on the Test set

In [26]:
df_test = (
    ebnerd_from_path(
        PATH.joinpath(PATH, "ebnerd_testset/test")
    )
    .sample(fraction=FRACTION)
)

print(f"Test samples: {df_test.height}")
print("Test Data Sample:")
print(df_test.head(2))


Test samples: 135367
Test Data Sample:
shape: (2, 15)
┌───────────┬───────────┬───────────┬───────────┬───┬───────────┬───────────┬───────────┬──────────┐
│ impressio ┆ impressio ┆ read_time ┆ scroll_pe ┆ … ┆ is_subscr ┆ session_i ┆ is_beyond ┆ article_ │
│ n_id      ┆ n_time    ┆ ---       ┆ rcentage  ┆   ┆ iber      ┆ d         ┆ _accuracy ┆ id_fixed │
│ ---       ┆ ---       ┆ f32       ┆ ---       ┆   ┆ ---       ┆ ---       ┆ ---       ┆ ---      │
│ u32       ┆ datetime[ ┆           ┆ f32       ┆   ┆ bool      ┆ u32       ┆ bool      ┆ list[i32 │
│           ┆ μs]       ┆           ┆           ┆   ┆           ┆           ┆           ┆ ]        │
╞═══════════╪═══════════╪═══════════╪═══════════╪═══╪═══════════╪═══════════╪═══════════╪══════════╡
│ 385779184 ┆ 2023-06-0 ┆ 9.0       ┆ null      ┆ … ┆ false     ┆ 84563669  ┆ false     ┆ [9785076 │
│           ┆ 3         ┆           ┆           ┆   ┆           ┆           ┆           ┆ ,        │
│           ┆ 23:52:49  ┆           ┆

In [27]:
df_test = (
    ebnerd_from_path(PATH.joinpath("ebnerd_testset", "test"), history_size=HISTORY_SIZE)
    .sample(fraction=FRACTION_TEST)
    .with_columns(
        pl.col(DEFAULT_INVIEW_ARTICLES_COL)
        .list.first()
        .alias(DEFAULT_CLICKED_ARTICLES_COL)
    )
    .select(COLUMNS + [DEFAULT_IS_BEYOND_ACCURACY_COL])
    .with_columns(
        pl.col(DEFAULT_INVIEW_ARTICLES_COL)
        .list.eval(pl.element() * 0)
        .alias(DEFAULT_LABELS_COL)
    )
)
df_test.head()

user_id,impression_id,impression_time,article_id_fixed,article_ids_clicked,article_ids_inview,is_beyond_accuracy,labels
u32,u32,datetime[μs],list[i32],i32,list[i32],bool,list[i32]
35982,6451339,2023-06-05 15:02:49,"[9786268, 9782806, … 9789494]",9796527,"[9796527, 7851321, … 9492777]",false,"[0, 0, … 0]"
36012,6451363,2023-06-05 15:03:56,"[9788323, 9788362, … 9790885]",9798532,"[9798532, 9791602, … 9798958]",false,"[0, 0, … 0]"
36162,6451382,2023-06-05 15:25:53,"[9788524, 9788106, … 9790700]",9798498,"[9798498, 9793856, … 9798724]",false,"[0, 0, … 0]"
36162,6451383,2023-06-05 15:26:35,"[9788524, 9788106, … 9790700]",9797419,"[9797419, 9798829, … 9798805]",false,"[0, 0, … 0]"
36162,6451385,2023-06-05 15:26:14,"[9788524, 9788106, … 9790700]",9785014,"[9785014, 9798958, … 9486080]",false,"[0, 0, … 0]"


# Take a subset of the test set for implementation

I break it so that I can eun the test

In [29]:
import polars as pl

# Assume df_test is already defined
df_test = df_test[:1000]  # Restrict to first 10,000 rows

# Split 500 rows for each case
df_false = df_test[:500].with_columns(
    pl.lit(False).alias("is_beyond_accuracy")
)

df_true = df_test[500:1000].with_columns(
    pl.lit(True).alias("is_beyond_accuracy")
)

# Combine into a single DataFrame
df_test = pl.concat([df_false, df_true])

# Verify the distribution
print(
    df_test.groupby("is_beyond_accuracy")
    .agg(pl.count().alias("count"))
)


shape: (2, 2)
┌────────────────────┬───────┐
│ is_beyond_accuracy ┆ count │
│ ---                ┆ ---   │
│ bool               ┆ u32   │
╞════════════════════╪═══════╡
│ true               ┆ 500   │
│ false              ┆ 500   │
└────────────────────┴───────┘


C:\Users\antot\AppData\Local\Temp\ipykernel_18860\4152766399.py:20: DeprecationWarning: `groupby` is deprecated. It has been renamed to `group_by`.
  df_test.groupby("is_beyond_accuracy")
C:\Users\antot\AppData\Local\Temp\ipykernel_18860\4152766399.py:21: DeprecationWarning: `pl.count()` is deprecated. Please use `pl.len()` instead.
  .agg(pl.count().alias("count"))


In [30]:
# Filter rows into two subsets
df_test_wo_beyond = df_test.filter(~pl.col("is_beyond_accuracy"))
df_test_w_beyond = df_test.filter(pl.col("is_beyond_accuracy"))

# Verify the split
print("Rows without beyond accuracy (False):", df_test_wo_beyond.shape[0])
print("Rows with beyond accuracy (True):", df_test_w_beyond.shape[0])


Rows without beyond accuracy (False): 500
Rows with beyond accuracy (True): 500


In [31]:
from ebrec.utils._polars import split_df_chunks



df_test_chunks = split_df_chunks(df_test_wo_beyond, n_chunks=N_CHUNKS_TEST)
df_pred_test_wo_beyond = []

In [32]:
BATCH_SIZE_TRAIN = 32
BATCH_SIZE_VAL = 32
BATCH_SIZE_TEST_WO_B = 32
BATCH_SIZE_TEST_W_B = 4
N_CHUNKS_TEST = 10
CHUNKS_DONE = 0

In [33]:
import gc
from tensorflow.keras.backend import clear_session
TEST_DF_DUMP = DUMP_DIR.joinpath("test_predictions", MODEL_NAME)
TEST_DF_DUMP.mkdir(parents=True, exist_ok=True)

df_test_chunks = split_df_chunks(df_test_wo_beyond, n_chunks=N_CHUNKS_TEST)
df_pred_test_wo_beyond = []

for i, df_test_chunk in enumerate(df_test_chunks[CHUNKS_DONE:], start=1 + CHUNKS_DONE):
    print(f"Init test-dataloader: {i}/{len(df_test_chunks)}")
    # Initialize DataLoader
    test_dataloader_wo_b = NRMSDataLoader(
        behaviors=df_test_chunk,
        article_dict=article_mapping_test,
        unknown_representation="zeros",
        history_column=DEFAULT_HISTORY_ARTICLE_ID_COL,
        eval_mode=True,
        batch_size=BATCH_SIZE_TEST_WO_B,
    )
    # Predict and clear session
    scores = model.scorer.predict(test_dataloader_wo_b)
    clear_session()

    # Process the predictions
    df_test_chunk = add_prediction_scores(df_test_chunk, scores.tolist()).with_columns(
        pl.col("scores")
        .map_elements(lambda x: list(rank_predictions_by_score(x)))
        .alias("ranked_scores")
    )

    # Save the processed chunk
    df_test_chunk.select(DEFAULT_IMPRESSION_ID_COL, "ranked_scores").write_parquet(
        TEST_DF_DUMP.joinpath(f"pred_wo_ba_{i}.parquet")
    )

    # Append and clean up
    df_pred_test_wo_beyond.append(df_test_chunk)

    # Cleanup
    del df_test_chunk, test_dataloader_wo_b, scores
    gc.collect()

Init test-dataloader: 1/10
2/2 [==============================] - 5s 2s/step
Init test-dataloader: 2/10
2/2 [==============================] - 4s 1s/step
Init test-dataloader: 3/10
2/2 [==============================] - 5s 961ms/step
Init test-dataloader: 4/10
2/2 [==============================] - 4s 1s/step
Init test-dataloader: 5/10
2/2 [==============================] - 4s 1s/step
Init test-dataloader: 6/10
2/2 [==============================] - 4s 1s/step
Init test-dataloader: 7/10
2/2 [==============================] - 4s 1s/step
Init test-dataloader: 8/10
2/2 [==============================] - 4s 1s/step
Init test-dataloader: 9/10
2/2 [==============================] - 4s 1s/step
Init test-dataloader: 10/10
2/2 [==============================] - 4s 1s/step


In [34]:
import polars as pl

# Concatenate all DataFrame chunks into a single DataFrame
df_pred_test_wo_beyond = pl.concat(df_pred_test_wo_beyond)

# Now you can use the .select() method
df_pred_test_wo_beyond.select(DEFAULT_IMPRESSION_ID_COL, "scores").write_parquet(
    TEST_DF_DUMP.joinpath("pred_wo_ba.parquet")
)

# View the head of the DataFrame
print(df_pred_test_wo_beyond.head(30))


shape: (30, 10)
┌─────────┬────────────┬───────────┬───────────┬───┬───────────┬───────────┬───────────┬───────────┐
│ user_id ┆ impression ┆ impressio ┆ article_i ┆ … ┆ is_beyond ┆ labels    ┆ scores    ┆ ranked_sc │
│ ---     ┆ _id        ┆ n_time    ┆ d_fixed   ┆   ┆ _accuracy ┆ ---       ┆ ---       ┆ ores      │
│ u32     ┆ ---        ┆ ---       ┆ ---       ┆   ┆ ---       ┆ list[i32] ┆ list[f64] ┆ ---       │
│         ┆ u32        ┆ datetime[ ┆ list[i32] ┆   ┆ bool      ┆           ┆           ┆ list[i64] │
│         ┆            ┆ μs]       ┆           ┆   ┆           ┆           ┆           ┆           │
╞═════════╪════════════╪═══════════╪═══════════╪═══╪═══════════╪═══════════╪═══════════╪═══════════╡
│ 35982   ┆ 6451339    ┆ 2023-06-0 ┆ [9786268, ┆ … ┆ false     ┆ [0, 0, …  ┆ [0.01932, ┆ [8, 2, …  │
│         ┆            ┆ 5         ┆ 9782806,  ┆   ┆           ┆ 0]        ┆ 0.132936, ┆ 7]        │
│         ┆            ┆ 15:02:49  ┆ …         ┆   ┆           ┆           

In [ ]:
print(type(df_pred_test_wo_beyond))


<class 'list'>


In [ ]:
# from ebrec.utils._constants import (
#     DEFAULT_HISTORY_ARTICLE_ID_COL,
#     DEFAULT_IS_BEYOND_ACCURACY_COL,
#     DEFAULT_CLICKED_ARTICLES_COL,
#     DEFAULT_INVIEW_ARTICLES_COL,
#     DEFAULT_IMPRESSION_ID_COL,
#     DEFAULT_SUBTITLE_COL,
#     DEFAULT_LABELS_COL,
#     DEFAULT_TITLE_COL,
#     DEFAULT_USER_COL,
# )

# # Prepare test data (without beyond-accuracy data)
# # df_pred_test_wo_beyond = pl.concat(df_pred_test_wo_beyond)



In [ ]:
# df_pred_test_wo_beyond.select(DEFAULT_IMPRESSION_ID_COL, "ranked_scores").write_parquet(
#     TEST_DF_DUMP.joinpath("pred_wo_ba.parquet")
# )

In [35]:
print("Init test-dataloader: beyond-accuracy")
test_dataloader_w_b = NRMSDataLoader(
    behaviors=df_test_w_beyond,
    article_dict=article_mapping_test,
    unknown_representation="zeros",
    history_column=DEFAULT_HISTORY_ARTICLE_ID_COL,
    eval_mode=True,
    batch_size=BATCH_SIZE_TEST_W_B,
)

Init test-dataloader: beyond-accuracy


In [36]:
scores = model.scorer.predict(test_dataloader_w_b)
df_pred_test_w_beyond = add_prediction_scores(
    df_test_w_beyond, scores.tolist()
).with_columns(
    pl.col("scores")
    .map_elements(lambda x: list(rank_predictions_by_score(x)))
    .alias("ranked_scores")
)
df_pred_test_w_beyond.select(DEFAULT_IMPRESSION_ID_COL, "ranked_scores").write_parquet(
    TEST_DF_DUMP.joinpath("pred_w_ba.parquet")
)

125/125 [==============================] - 43s 340ms/step


In [37]:
# Check the schemas of both DataFrames
print("Schema of df_pred_test_wo_beyond:")
print(df_pred_test_wo_beyond.schema)

print("Schema of df_pred_test_w_beyond:")
print(df_pred_test_w_beyond.schema)

Schema of df_pred_test_wo_beyond:
OrderedDict([('user_id', UInt32), ('impression_id', UInt32), ('impression_time', Datetime(time_unit='us', time_zone=None)), ('article_id_fixed', List(Int32)), ('article_ids_clicked', Int32), ('article_ids_inview', List(Int32)), ('is_beyond_accuracy', Boolean), ('labels', List(Int32)), ('scores', List(Float64)), ('ranked_scores', List(Int64))])
Schema of df_pred_test_w_beyond:
OrderedDict([('user_id', UInt32), ('impression_id', UInt32), ('impression_time', Datetime(time_unit='us', time_zone=None)), ('article_id_fixed', List(Int32)), ('article_ids_clicked', Int32), ('article_ids_inview', List(Int32)), ('is_beyond_accuracy', Boolean), ('labels', List(Int32)), ('scores', List(Float64)), ('ranked_scores', List(Int64))])


In [38]:
# Check the schemas of both DataFrames
print("Schema of df_pred_test_wo_beyond:")
print(df_pred_test_wo_beyond.schema)

print("Schema of df_pred_test_w_beyond:")
print(df_pred_test_w_beyond.schema)

# Align column types
df_pred_test_wo_beyond = df_pred_test_wo_beyond.with_columns(
    [pl.col(column).cast(df_pred_test_w_beyond.schema[column]) for column in df_pred_test_w_beyond.schema]
)

# Combine both DataFrames
df_test = pl.concat([df_pred_test_wo_beyond, df_pred_test_w_beyond])

# Write to Parquet
df_test.select(DEFAULT_IMPRESSION_ID_COL, "ranked_scores").write_parquet(
    TEST_DF_DUMP.joinpath("pred_concat.parquet")
)


Schema of df_pred_test_wo_beyond:
OrderedDict([('user_id', UInt32), ('impression_id', UInt32), ('impression_time', Datetime(time_unit='us', time_zone=None)), ('article_id_fixed', List(Int32)), ('article_ids_clicked', Int32), ('article_ids_inview', List(Int32)), ('is_beyond_accuracy', Boolean), ('labels', List(Int32)), ('scores', List(Float64)), ('ranked_scores', List(Int64))])
Schema of df_pred_test_w_beyond:
OrderedDict([('user_id', UInt32), ('impression_id', UInt32), ('impression_time', Datetime(time_unit='us', time_zone=None)), ('article_id_fixed', List(Int32)), ('article_ids_clicked', Int32), ('article_ids_inview', List(Int32)), ('is_beyond_accuracy', Boolean), ('labels', List(Int32)), ('scores', List(Float64)), ('ranked_scores', List(Int64))])


In [39]:
print(type(df_pred_test_wo_beyond))
print(type(df_pred_test_w_beyond))


<class 'polars.dataframe.frame.DataFrame'>
<class 'polars.dataframe.frame.DataFrame'>


In [40]:
df_test.head

<bound method DataFrame.head of shape: (1_000, 10)
┌─────────┬────────────┬───────────┬───────────┬───┬───────────┬───────────┬───────────┬───────────┐
│ user_id ┆ impression ┆ impressio ┆ article_i ┆ … ┆ is_beyond ┆ labels    ┆ scores    ┆ ranked_sc │
│ ---     ┆ _id        ┆ n_time    ┆ d_fixed   ┆   ┆ _accuracy ┆ ---       ┆ ---       ┆ ores      │
│ u32     ┆ ---        ┆ ---       ┆ ---       ┆   ┆ ---       ┆ list[i32] ┆ list[f64] ┆ ---       │
│         ┆ u32        ┆ datetime[ ┆ list[i32] ┆   ┆ bool      ┆           ┆           ┆ list[i64] │
│         ┆            ┆ μs]       ┆           ┆   ┆           ┆           ┆           ┆           │
╞═════════╪════════════╪═══════════╪═══════════╪═══╪═══════════╪═══════════╪═══════════╪═══════════╡
│ 35982   ┆ 6451339    ┆ 2023-06-0 ┆ [9786268, ┆ … ┆ false     ┆ [0, 0, …  ┆ [0.01932, ┆ [8, 2, …  │
│         ┆            ┆ 5         ┆ 9782806,  ┆   ┆           ┆ 0]        ┆ 0.132936, ┆ 7]        │
│         ┆            ┆ 15:02:49  ┆ …  

In [41]:
import polars as pl
import numpy as np

# Update the 'labels' column to match the length of 'ranked_scores' and assign 1 to the highest rank
df_test = df_test.with_columns(
    pl.struct(["ranked_scores", "scores"])
    .apply(lambda row: [1 if rank == 1 else 0 for rank in row["ranked_scores"]]
           if len(row["ranked_scores"]) == len(row["scores"]) else None)
    .alias("labels")
)

# Check for rows where labels are None (mismatched lengths)
invalid_rows = df_test.filter(pl.col("labels").is_null())

if invalid_rows.height > 0:
    print("Found rows with mismatched 'ranked_scores' and 'scores':")
    print(invalid_rows)

# Verify the updated 'labels' column
print(df_test.select(["ranked_scores", "labels"]))


shape: (1_000, 2)
┌────────────────┬─────────────┐
│ ranked_scores  ┆ labels      │
│ ---            ┆ ---         │
│ list[i64]      ┆ list[i64]   │
╞════════════════╪═════════════╡
│ [8, 2, … 7]    ┆ [0, 0, … 0] │
│ [6, 2, … 3]    ┆ [0, 0, … 0] │
│ [5, 4, … 3]    ┆ [0, 0, … 0] │
│ [11, 7, … 6]   ┆ [0, 0, … 0] │
│ [2, 1, … 7]    ┆ [0, 1, … 0] │
│ …              ┆ …           │
│ [3, 28, … 14]  ┆ [0, 0, … 0] │
│ [20, 19, … 28] ┆ [0, 0, … 0] │
│ [2, 4, … 1]    ┆ [0, 0, … 1] │
│ [1, 4, … 5]    ┆ [1, 0, … 0] │
│ [30, 31, … 13] ┆ [0, 0, … 0] │
└────────────────┴─────────────┘


C:\Users\antot\AppData\Local\Temp\ipykernel_18860\247236528.py:7: DeprecationWarning: `apply` is deprecated. It has been renamed to `map_elements`.
  .apply(lambda row: [1 if rank == 1 else 0 for rank in row["ranked_scores"]]


In [60]:
df_test.head()

user_id,impression_id,impression_time,article_id_fixed,article_ids_clicked,article_ids_inview,is_beyond_accuracy,labels,scores,ranked_scores
u32,u32,datetime[μs],list[i32],i32,list[i32],bool,list[i64],list[f64],list[i64]
35982,6451339,2023-06-05 15:02:49,"[9786268, 9782806, … 9789494]",9796527,"[9796527, 7851321, … 9492777]",false,"[0, 0, … 0]","[0.630941, 0.950646, … 0.008185]","[3, 2, … 9]"
36012,6451363,2023-06-05 15:03:56,"[9788323, 9788362, … 9790885]",9798532,"[9798532, 9791602, … 9798958]",false,"[0, 0, … 0]","[0.104358, 0.001216, … 0.179523]","[4, 7, … 3]"
36162,6451382,2023-06-05 15:25:53,"[9788524, 9788106, … 9790700]",9798498,"[9798498, 9793856, … 9798724]",false,"[0, 0, … 0]","[0.097785, 0.000392, … 0.015833]","[2, 5, … 3]"
36162,6451383,2023-06-05 15:26:35,"[9788524, 9788106, … 9790700]",9797419,"[9797419, 9798829, … 9798805]",false,"[0, 0, … 0]","[0.024571, 0.001634, … 0.580246]","[9, 11, … 3]"
36162,6451385,2023-06-05 15:26:14,"[9788524, 9788106, … 9790700]",9785014,"[9785014, 9798958, … 9486080]",false,"[0, 0, … 1]","[0.918962, 0.120063, … 0.9671]","[2, 6, … 1]"


-----------------------------------------------------------------------------

This is using the validation, simply add the testset to your flow.

In [ ]:
DATASPLIT = "ebnerd_large_test"

write_submission_file(
    impression_ids=df_test[DEFAULT_IMPRESSION_ID_COL],
    prediction_scores=df_test["ranked_scores"],
    path=DUMP_DIR.joinpath("predictions.txt"),
    filename_zip=f"{DATASPLIT}_predictions-{MODEL_NAME}.zip",
)

0it [00:00, ?it/s]

2446it [00:00, 27609.70it/s]

Zipping ebnerd_predictions/predictions.txt to ebnerd_predictions/ebnerd_small_predictions-NRMSModel.zip
